In [1]:
from mtg_draftbot.draftbot.draftbot_sim import *
from draftbot_sim_adapted import MultiStratDraft
import sqlite3
import pandas as pd
import numpy as np
import json
import os

In [2]:
draft_path = 'C:/Users/trist/mtg_research/test_draft.txt'
cards_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/m20/m20-cards.json'
weights_path = 'C:/Users/trist/mtg_research/mtg_draftbot/data/new_generated_weights.json'
#Paths to the db files holding our outputs
ss_db_path = 'SingleStrat.sqlite'
ms_db_path = 'MultiStrat.sqlite'

In [3]:
draft_ss = Draft(n_drafters=8,
        n_rounds=3,
        n_cards_in_pack=14,
        cards_path=cards_path,
        card_values_path=weights_path)

draft_ms = MultiStratDraft(n_drafters_strat_one=2,
        strat_one='naive passes',
        n_drafters_strat_two=6,
        n_rounds=3,
        n_cards_in_pack=14,
        cards_path=cards_path,
        card_values_path=weights_path,
        packs_input_file=draft_path)

In [4]:
draft_ss.draft()
draft_ss.write_to_database(ss_db_path)
if os.path.exists(draft_path):
    os.remove(draft_path)
save_packs_array(draft_ss,draft_path)
draft_ms.draft()
draft_ms.write_to_database(ms_db_path)

C:\Users\trist\AppData\Local\Programs\Python\Python38\lib\site-packages\pandas\core\generic.py:2779: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [5]:
ss_conn = sqlite3.connect(ss_db_path)
ms_conn = sqlite3.connect(ms_db_path)

ss_picks_df = pd.read_sql("Select * From Picks",ss_conn)
ss_options_df = pd.read_sql("Select * From Options",ss_conn)
#ss_passes_df = pd.read_sql("Select * From Passes",ss_conn)
ss_prefs_df = pd.read_sql("Select * From Preferences",ss_conn)

In [6]:
ms_picks_df = pd.read_sql("Select * From Picks",ms_conn)
ms_options_df = pd.read_sql("Select * From Options",ms_conn)
ms_pass_df = pd.read_sql("Select * From Passes",ms_conn)
ms_prefs_df = pd.read_sql("Select * From Preferences",ms_conn)

In [7]:
mpdf = ms_picks_df.drop(columns=['pick_number','drafter','draft_id'])
spdf = ss_picks_df.drop(columns=['pick_number','drafter','draft_id'])


msof = ms_options_df.drop(columns=['pick_number','drafter','draft_id'])
ssof = ss_options_df.drop(columns=['pick_number','drafter','draft_id'])

ms_pass_df = ms_pass_df.drop(columns=['pick_number','drafter','draft_id'])

ms_prefs_df = ms_prefs_df.drop(columns=['pick_number','drafter','draft_id'])
ss_prefs_df = ss_prefs_df.drop(columns=['pick_number','drafter','draft_id'])

In [8]:


pick_to_explore = 10

print(np.where(mpdf.iloc[pick_to_explore]==1))
print(np.where(msof.iloc[pick_to_explore]==1))
print(np.where(np.sum(ms_pass_df.iloc[0:pick_to_explore],axis=0)!=0))

#print(ms_prefs_df.iloc[35])

print(np.where(spdf.iloc[pick_to_explore]==1))
print(np.where(ssof.iloc[pick_to_explore]==1))
#print(ss_prefs_df.iloc[35])

(array([242], dtype=int64),)
(array([ 74, 202, 223, 242], dtype=int64),)
(array([  0,  11,  23,  27,  30,  35,  43,  49,  66,  74,  77,  93,  96,
        97, 101, 103, 106, 107, 110, 118, 121, 128, 132, 134, 142, 148,
       153, 156, 172, 173, 181, 189, 190, 193, 198, 201, 202, 205, 210,
       215, 219, 220, 222, 223, 234, 242, 243, 245, 247, 260, 269, 274,
       275, 276, 278, 284, 285, 287, 289, 301], dtype=int64),)
(array([202], dtype=int64),)
(array([ 74, 202, 223, 242], dtype=int64),)


In [9]:
ms_conn.close()
ss_conn.close()